In [18]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing
from sklearn import utils
from sklearn import ensemble
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression

In [2]:
test_table=pd.read_csv('test.csv')
train_table=pd.read_csv('train.csv')

In [3]:
categorical_columns_test = [c for c in test_table.columns if test_table[c].dtype.name == 'object']
numerical_columns_test   = [c for c in test_table.columns if test_table[c].dtype.name != 'object']
categorical_columns_train = [c for c in train_table.columns if train_table[c].dtype.name == 'object']
numerical_columns_train   = [c for c in train_table.columns if train_table[c].dtype.name != 'object']

In [4]:
test_table = test_table.fillna(test_table.median(axis=0), axis=0)
train_table = train_table.fillna(train_table.median(axis=0), axis=0)
train_table_describe = train_table.describe(include=[object])
test_table_describe = test_table.describe(include=[object])

In [5]:
binary_columns_train    = [c for c in categorical_columns_train if train_table_describe[c]['unique'] == 2]
nonbinary_columns_train = [c for c in categorical_columns_train if train_table_describe[c]['unique'] > 2]
binary_columns_test    = [c for c in categorical_columns_test if test_table_describe[c]['unique'] == 2]
nonbinary_columns_test = [c for c in categorical_columns_test if test_table_describe[c]['unique'] > 2]

In [6]:
for c in binary_columns_test[1:]:
    top = test_table_describe[c]['top']
    top_items = test_table[c] == top
    test_table.loc[top_items, c] = 0
    test_table.loc[np.logical_not(top_items), c] = 1
for c in binary_columns_train[1:]:
    top = train_table_describe[c]['top']
    top_items = train_table[c] == top
    train_table.loc[top_items, c] = 0
    train_table.loc[np.logical_not(top_items), c] = 1

In [7]:
test_nonbinary = pd.get_dummies(test_table[nonbinary_columns_test])
train_nonbinary = pd.get_dummies(train_table[nonbinary_columns_train])

In [8]:
test = pd.concat((test_table[numerical_columns_test], test_table[binary_columns_test], test_nonbinary), axis=1)
train = pd.concat((train_table[numerical_columns_train], train_table[binary_columns_train], train_nonbinary), axis=1)

In [9]:
test=test.drop("Street",axis=1)
train=train.drop("Street", axis=1)

In [30]:
test=test.drop(['PoolQC'],axis=1)

In [10]:
train_y=train.SalePrice
train_x=train.drop('SalePrice', axis=1)
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size = 0.3, random_state = 11)

In [11]:
iowa_model = RandomForestClassifier(n_estimators=1000, random_state=11)
iowa_model.fit(X_train, y_train)
val_predictions = iowa_model.predict(X_test)

In [12]:
mean_absolute_error(val_predictions, y_test)

22516.73287671233

In [13]:
iowa_model = RandomForestClassifier()
iowa_model.fit(X_train, y_train)
val_predictions = iowa_model.predict(X_test)

/home/dmitriy/.local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [14]:
mean_absolute_error(val_predictions, y_test)

43856.43378995434

In [15]:
gbt = ensemble.GradientBoostingClassifier(n_estimators=100, random_state=11)
gbt.fit(X_train, y_train)
val_predictions = gbt.predict(X_test)
mean_absolute_error(val_predictions, y_test)

45005.77168949772

In [21]:
reg = LinearRegression().fit(X_train, y_train)
val_predictions = reg.predict(X_test)
mean_absolute_error(val_predictions, y_test)
#Best one

18655.48616751691

In [31]:
reg = LinearRegression().fit(train_x[test.columns], train_y)
val_predictions = reg.predict(test)

In [32]:
df = pd.DataFrame({'Id': test_table.Id, 'SalePrice': val_predictions})
df.to_csv('qwe.csv', encoding='utf-8', index=False)